
## PyZX Hybrid Mixed Quantum-Classical Circuit Demo


This demo showcases PyZX's support for hybrid quantum-classical circuits using
the 'ground' feature. This allows modeling of measurements, classical control,
and mixed quantum-classical computation within the ZX-calculus framework.

The 'ground' feature in PyZX represents classical information flow and control,
enabling the representation of:
- Quantum measurements that produce classical bits
- Classical control of quantum gates (conditional operations)
- Mixed quantum-classical algorithms
- Teleportation and other protocols involving classical communication

1. Quantum Error Correction with Syndrome Extraction

Demonstrates classical syndrome processing from quantum measurements
Shows how ground vertices handle error correction decision-making
Models realistic QEC protocols with classical control loops

2. Adaptive Quantum Algorithms

Mid-circuit measurements that dynamically change circuit execution
Classical decision nodes that control subsequent quantum operations
Multiple execution paths based on measurement outcomes

3. Quantum Machine Learning Pipeline

Classical feature preprocessing feeding into quantum encoding
Hybrid variational layers with classical parameter control
Classical postprocessing for gradient computation and optimization

4. Advanced Mid-Circuit Measurement Patterns

Sequential measurements with feed-forward control
Parallel measurements with joint classical processing
Correlation analysis between measurement outcomes

5. Ground Preservation Analysis

Comprehensive testing of how ground properties survive graph operations
Verification that hybrid nature is maintained through transformations
Composition rules for combining quantum and classical circuits

In [6]:
import pyzx as zx
from fractions import Fraction
import numpy as np
from typing import List, Tuple, Dict, Set

 In traditional ZX-calculus, graphs model quantum operations using Z and X spiders ```(nodes)``` connected by edges. This script extends that concept using "grounded" vertices, which represent classical information, such as measurement outcomes or classically controlled operations. By setting ```ground=True``` on certain vertices, PyZX can distinguish quantum behavior from classical control, enabling modeling of hybrid algorithms, such as quantum teleportation, conditional gates, and variational circuits.

Mathematically, grounding a vertex allows it to behave like a classical node in a computational graph, thus mixing quantum operations (unitary evolution) with non-unitary classical branching or feedback. The demo walks through six structured examples: (1) basic ground operations, (2) measurement circuits, (3) classically controlled gates, (4) quantum teleportation via classical bits, (5) how grounded vertices affect simplification, and (6) a hybrid variational algorithm combining classical parameter optimization with quantum circuits. Each example uses ZX-graph construction and PyZX's hybrid utilities like is_ground, ```grounds()```, and ```is_hybrid()``` to show how measurement, control, and classical logic are naturally incorporated into the quantum circuit graph structure. This demonstrates that PyZX's graphical model can support full hybrid quantum-classical computation rather than just unitary-only quantum circuits.

Let’s see how PyZX can be used to model hybrid quantum-classical circuits using its ground feature. This tutorial-style code walks through six core demonstrations using ```zx.Graph()``` objects—PyZX’s underlying graphical representation of quantum circuits, based on ZX-calculus. The key idea is that by marking certain vertices with ```ground=True```, we can distinguish classical information (like measurement outcomes or classical control signals) from purely quantum operations.

- Basic Ground Operations show how to create and inspect ground vertices.

- Measurement Circuits use ground vertices to represent quantum measurements (where quantum data collapses into classical bits).

- Conditional Gates demonstrate how classical bits can control quantum operations (e.g., applying an X gate based on a classical condition).

- Teleportation Protocol encodes a full quantum teleportation algorithm with classical communication channels modeled as ground vertices.

- Circuit Reduction highlights that ground-connected vertices affect how simplifications or adjoints of circuits are performed.

- Hybrid Algorithm Example combines classical parameters (from an optimizer) with a quantum circuit—mirroring how variational quantum algorithms work.

Mathematically, this models a hybrid computation graph: quantum operations remain unitary until measured, and classical control is handled via ground vertices that propagate classical bits through the graph. The PyZX API provides tools like is_ground, grounds, and is_hybrid to analyze and manage these mixed-mode systems. This hybrid extension enriches ZX-diagrams beyond pure quantum logic, enabling simulations of full hybrid algorithms such as variational circuits, feedback loops, and teleportation—all within one unified graph model.

In [8]:
class HybridCircuitDemo:
    """Demonstrates hybrid quantum-classical circuits using PyZX grounds."""

    def __init__(self):
        """Initialize the demo with basic setup."""
        print("=== PyZX Hybrid Quantum-Classical Circuit Demo ===\n")

    def basic_ground_operations(self):
        """Demonstrate basic ground vertex operations."""
        print("1. Basic Ground Operations")
        print("-" * 30)

        # Create a simple graph
        g = zx.Graph()

        # Add some vertices
        v1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1)
        v2 = g.add_vertex(zx.VertexType.X, qubit=1, row=1)
        v3 = g.add_vertex(zx.VertexType.Z, qubit=0, row=2)

        print(f"Created graph with {g.num_vertices()} vertices")

        # Set ground connections
        g.set_ground(v1, True)
        g.set_ground(v3, True)

        print(f"Vertex {v1} is ground-connected: {g.is_ground(v1)}")
        print(f"Vertex {v2} is ground-connected: {g.is_ground(v2)}")
        print(f"Vertex {v3} is ground-connected: {g.is_ground(v3)}")

        # Get all ground vertices
        ground_vertices = g.grounds()
        print(f"Ground vertices: {ground_vertices}")

        # Check if graph is hybrid
        print(f"Graph is hybrid (has grounds): {g.is_hybrid()}")

        print()

    def measurement_circuit(self):
        """Demonstrate a circuit with quantum measurements."""
        print("2. Quantum Measurement Circuit")
        print("-" * 30)

        # Create a circuit that prepares a Bell state and measures both qubits
        g = zx.Graph()

        # Add input/output boundaries
        input1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)
        input2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)

        # Hadamard on first qubit (represented as Z-spider with phase π/2)
        h_gate = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, phase=Fraction(1,2))

        # CNOT gate (represented as connected Z and X spiders)
        cnot_control = g.add_vertex(zx.VertexType.Z, qubit=0, row=2)
        cnot_target = g.add_vertex(zx.VertexType.X, qubit=1, row=2)

        # Measurement vertices (ground-connected to represent classical output)
        measure1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=3, ground=True)
        measure2 = g.add_vertex(zx.VertexType.Z, qubit=1, row=3, ground=True)

        # Classical outputs
        output1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=4)
        output2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=4)

        # Connect the circuit
        g.add_edge((input1, h_gate))
        g.add_edge((input2, cnot_target))
        g.add_edge((h_gate, cnot_control))
        g.add_edge((cnot_control, cnot_target))  # CNOT connection
        g.add_edge((cnot_control, measure1))
        g.add_edge((cnot_target, measure2))
        g.add_edge((measure1, output1))
        g.add_edge((measure2, output2))

        # Set inputs/outputs
        g.set_inputs((input1, input2))
        g.set_outputs((output1, output2))

        print(f"Bell state measurement circuit created")
        print(f"Number of vertices: {g.num_vertices()}")
        print(f"Number of ground vertices: {len(g.grounds())}")
        print(f"Ground vertices: {g.grounds()}")
        print(f"Circuit is hybrid: {g.is_hybrid()}")

        # Analyze the measurement structure
        print("\nMeasurement Analysis:")
        for v in g.grounds():
            neighbors = g.neighbors(v)
            print(f"  Ground vertex {v}: connected to {neighbors}")
            print(f"  Type: {g.type(v)}, Phase: {g.phase(v)}")

        print()

    def conditional_gate_circuit(self):
        """Demonstrate classical control of quantum gates."""
        print("3. Classical Control Circuit")
        print("-" * 30)

        g = zx.Graph()

        # Create a circuit where a quantum gate is controlled by a classical bit
        # This models: if (classical_bit) then apply_X_gate()

        # Quantum input
        q_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Classical control input (ground-connected)
        c_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0, ground=True)

        # Control logic vertex (processes classical information)
        control_logic = g.add_vertex(zx.VertexType.Z, qubit=1, row=1, ground=True)

        # Controlled quantum gate (X gate, controlled by classical bit)
        controlled_x = g.add_vertex(zx.VertexType.X, qubit=0, row=2, phase=Fraction(1))

        # Control connection vertex (mediates classical-quantum interaction)
        control_conn = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Quantum output
        q_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=3)

        # Classical output (copy of control bit)
        c_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=3, ground=True)

        # Connect the circuit
        g.add_edge((q_input, control_conn))
        g.add_edge((c_input, control_logic))
        g.add_edge((control_logic, control_conn))  # Classical control
        g.add_edge((control_conn, controlled_x))
        g.add_edge((controlled_x, q_output))
        g.add_edge((control_logic, c_output))  # Classical output

        g.set_inputs((q_input, c_input))
        g.set_outputs((q_output, c_output))

        print(f"Conditional gate circuit created")
        print(f"Hybrid circuit: {g.is_hybrid()}")
        print(f"Ground vertices: {len(g.grounds())}")

        # Analyze classical control structure
        print("\nClassical Control Analysis:")
        classical_vertices = g.grounds()
        for v in classical_vertices:
            neighbors = [n for n in g.neighbors(v)]
            quantum_neighbors = [n for n in neighbors if not g.is_ground(n)]
            classical_neighbors = [n for n in neighbors if g.is_ground(n)]

            print(f"  Classical vertex {v}:")
            print(f"    Quantum connections: {quantum_neighbors}")
            print(f"    Classical connections: {classical_neighbors}")

        print()

    def teleportation_protocol(self):
        """Demonstrate quantum teleportation with classical communication."""
        print("4. Quantum Teleportation Protocol")
        print("-" * 30)

        g = zx.Graph()

        # Teleportation involves:
        # 1. Unknown quantum state to be teleported
        # 2. Entangled Bell pair shared between Alice and Bob
        # 3. Alice's Bell measurement (produces 2 classical bits)
        # 4. Bob's conditional operations based on classical bits

        # Alice's unknown state input
        alice_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Bell pair inputs (|00⟩ + |11⟩)/√2
        bell_alice = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)
        bell_bob = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=0)

        # Bell pair preparation
        h_bell = g.add_vertex(zx.VertexType.Z, qubit=1, row=1, phase=Fraction(1,2))
        cnot_bell_ctrl = g.add_vertex(zx.VertexType.Z, qubit=1, row=2)
        cnot_bell_targ = g.add_vertex(zx.VertexType.X, qubit=2, row=2)

        # Alice's Bell measurement
        alice_cnot_ctrl = g.add_vertex(zx.VertexType.Z, qubit=0, row=3)
        alice_cnot_targ = g.add_vertex(zx.VertexType.X, qubit=1, row=3)
        alice_h = g.add_vertex(zx.VertexType.Z, qubit=0, row=4, phase=Fraction(1,2))

        # Alice's measurement outcomes (classical bits)
        measure_x = g.add_vertex(zx.VertexType.Z, qubit=0, row=5, ground=True)
        measure_z = g.add_vertex(zx.VertexType.Z, qubit=1, row=5, ground=True)

        # Classical communication to Bob
        comm_x = g.add_vertex(zx.VertexType.Z, qubit=0, row=6, ground=True)
        comm_z = g.add_vertex(zx.VertexType.Z, qubit=1, row=6, ground=True)

        # Bob's conditional operations
        bob_x_gate = g.add_vertex(zx.VertexType.X, qubit=2, row=7, phase=Fraction(1))
        bob_z_gate = g.add_vertex(zx.VertexType.Z, qubit=2, row=8, phase=Fraction(1))

        # Bob's output (reconstructed state)
        bob_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=9)

        # Connect Bell pair preparation
        g.add_edge((bell_alice, h_bell))
        g.add_edge((bell_bob, cnot_bell_targ))
        g.add_edge((h_bell, cnot_bell_ctrl))
        g.add_edge((cnot_bell_ctrl, cnot_bell_targ))

        # Connect Alice's operations
        g.add_edge((alice_input, alice_cnot_ctrl))
        g.add_edge((cnot_bell_ctrl, alice_cnot_targ))
        g.add_edge((alice_cnot_ctrl, alice_cnot_targ))
        g.add_edge((alice_cnot_ctrl, alice_h))
        g.add_edge((alice_h, measure_x))
        g.add_edge((alice_cnot_targ, measure_z))

        # Classical communication
        g.add_edge((measure_x, comm_x))
        g.add_edge((measure_z, comm_z))

        # Bob's conditional operations (classical control)
        g.add_edge((cnot_bell_targ, bob_x_gate))
        g.add_edge((comm_x, bob_x_gate))  # Classical control
        g.add_edge((bob_x_gate, bob_z_gate))
        g.add_edge((comm_z, bob_z_gate))  # Classical control
        g.add_edge((bob_z_gate, bob_output))

        g.set_inputs((alice_input, bell_alice, bell_bob))
        g.set_outputs((bob_output,))

        print(f"Teleportation protocol circuit created")
        print(f"Total vertices: {g.num_vertices()}")
        print(f"Ground (classical) vertices: {len(g.grounds())}")
        print(f"Classical communication channels: {len([v for v in g.grounds() if 'comm' in str(v)])}")

        # Analyze information flow
        print("\nInformation Flow Analysis:")
        print("Classical vertices and their roles:")
        ground_vertices = list(g.grounds())
        for i, v in enumerate(ground_vertices):
            neighbors = g.neighbors(v)
            if any('measure' in str(n) for n in neighbors):
                print(f"  Vertex {v}: Measurement outcome")
            elif any('comm' in str(n) for n in neighbors):
                print(f"  Vertex {v}: Classical communication")
            else:
                print(f"  Vertex {v}: Classical control")

        print()

    def reduction_with_grounds(self):
        """Demonstrate how ground vertices affect circuit reduction."""
        print("5. Circuit Reduction with Grounds")
        print("-" * 30)

        # Create a simple circuit with some redundancy
        g = zx.Graph()

        input1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Identity-like structure that should be reducible
        z1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, phase=0)
        z2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=2, phase=0)

        # But one vertex is ground-connected (measurement)
        measure = g.add_vertex(zx.VertexType.Z, qubit=0, row=3, ground=True)

        output1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=4)

        g.add_edge((input1, z1))
        g.add_edge((z1, z2))
        g.add_edge((z2, measure))
        g.add_edge((measure, output1))

        g.set_inputs((input1,))
        g.set_outputs((output1,))

        print("Original circuit:")
        print(f"  Vertices: {g.num_vertices()}")
        print(f"  Edges: {g.num_edges()}")
        print(f"  Ground vertices: {len(g.grounds())}")
        print(f"  Is hybrid: {g.is_hybrid()}")

        # Show that grounds are preserved during operations
        g_copy = g.copy()
        print(f"\nAfter copying:")
        print(f"  Ground vertices preserved: {len(g_copy.grounds()) == len(g.grounds())}")
        print(f"  Ground vertices: {g_copy.grounds()}")

        # Demonstrate adjoint with grounds
        g_adj = g.adjoint()
        print(f"\nAfter taking adjoint:")
        print(f"  Ground vertices: {len(g_adj.grounds())}")
        print(f"  Still hybrid: {g_adj.is_hybrid()}")

        print()

    def hybrid_algorithm_example(self):
        """Show a complete hybrid quantum-classical algorithm."""
        print("6. Hybrid Algorithm Example: Quantum-Classical Feedback")
        print("-" * 50)

        # Simulate a variational quantum algorithm with classical optimization
        g = zx.Graph()

        # Quantum register
        q_inputs = []
        for i in range(3):
            q_inputs.append(g.add_vertex(zx.VertexType.BOUNDARY, qubit=i, row=0))

        # Classical parameter inputs (from optimizer)
        c_params = []
        for i in range(2):
            c_params.append(g.add_vertex(zx.VertexType.BOUNDARY, qubit=3+i, row=0, ground=True))

        # Parameterized quantum circuit
        # Layer 1: RY gates controlled by classical parameters
        ry_gates = []
        for i in range(3):
            ry = g.add_vertex(zx.VertexType.Z, qubit=i, row=1, phase=Fraction(1,4))
            param_ctrl = g.add_vertex(zx.VertexType.Z, qubit=3, row=1, ground=True)
            ry_gates.append(ry)
            g.add_edge((q_inputs[i], ry))
            g.add_edge((c_params[0], param_ctrl))
            g.add_edge((param_ctrl, ry))  # Classical control of gate parameter

        # Layer 2: Entangling gates
        entangling = []
        for i in range(2):
            cnot_ctrl = g.add_vertex(zx.VertexType.Z, qubit=i, row=2)
            cnot_targ = g.add_vertex(zx.VertexType.X, qubit=i+1, row=2)
            entangling.extend([cnot_ctrl, cnot_targ])
            g.add_edge((ry_gates[i], cnot_ctrl))
            g.add_edge((ry_gates[i+1], cnot_targ))
            g.add_edge((cnot_ctrl, cnot_targ))

        # Measurements for expectation value estimation
        measurements = []
        for i in range(3):
            measure = g.add_vertex(zx.VertexType.Z, qubit=i, row=3, ground=True)
            measurements.append(measure)
            if i < 2:
                g.add_edge((entangling[2*i], measure))
            else:
                g.add_edge((ry_gates[i], measure))

        # Classical processing of measurement results
        classical_proc = g.add_vertex(zx.VertexType.Z, qubit=5, row=4, ground=True, phase=0)
        for m in measurements:
            g.add_edge((m, classical_proc))

        # Classical outputs (cost function value, updated parameters)
        cost_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=5, row=5, ground=True)
        param_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=6, row=5, ground=True)

        g.add_edge((classical_proc, cost_output))
        g.add_edge((c_params[1], param_output))

        g.set_inputs(tuple(q_inputs + c_params))
        g.set_outputs((cost_output, param_output))

        print(f"Hybrid variational algorithm circuit:")
        print(f"  Total vertices: {g.num_vertices()}")
        print(f"  Quantum vertices: {g.num_vertices() - len(g.grounds())}")
        print(f"  Classical vertices: {len(g.grounds())}")
        print(f"  Classical inputs: {len([v for v in c_params])}")
        print(f"  Classical outputs: 2")

        # Analyze the hybrid structure
        print(f"\nHybrid Structure Analysis:")
        print(f"  Classical parameter control: {len([v for v in g.grounds() if g.row(v) == 1])}")
        print(f"  Quantum measurements: {len(measurements)}")
        print(f"  Classical processing nodes: 1")
        print(f"  Feedback loops: Classical params → Quantum gates → Measurements → Classical processing")

        print()

    def run_all_demos(self):
        """Run all demonstration functions."""
        self.basic_ground_operations()
        self.measurement_circuit()
        self.conditional_gate_circuit()
        self.teleportation_protocol()
        self.reduction_with_grounds()
        self.hybrid_algorithm_example()



def main():
    """Main function to run the demo."""
    try:
        demo = HybridCircuitDemo()
        demo.run_all_demos()
    except ImportError as e:
        print(f"Error: PyZX not found. Please install PyZX first: pip install pyzx")
        print(f"ImportError details: {e}")
    except Exception as e:
        print(f"Error running demo: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


=== PyZX Hybrid Quantum-Classical Circuit Demo ===

1. Basic Ground Operations
------------------------------
Created graph with 3 vertices
Vertex 0 is ground-connected: True
Vertex 1 is ground-connected: False
Vertex 2 is ground-connected: True
Ground vertices: {0, 2}
Graph is hybrid (has grounds): True

2. Quantum Measurement Circuit
------------------------------
Bell state measurement circuit created
Number of vertices: 9
Number of ground vertices: 2
Ground vertices: {5, 6}
Circuit is hybrid: True

Measurement Analysis:
  Ground vertex 5: connected to dict_keys([3, 7])
  Type: 1, Phase: 0
  Ground vertex 6: connected to dict_keys([4, 8])
  Type: 1, Phase: 0

3. Classical Control Circuit
------------------------------
Conditional gate circuit created
Hybrid circuit: True
Ground vertices: 4

Classical Control Analysis:
  Classical vertex 1:
    Quantum connections: []
    Classical connections: [2]
  Classical vertex 2:
    Quantum connections: []
    Classical connections: [1, 4, 6]

Let’s see how to build a simple classical circuit using the ZX-calculus with the `pyzx` library. In this introductory example, we construct a linear ZX-diagram that represents a basic classical signal flow using *grounded vertices*, which indicate classical (non-quantum) behavior. We start by creating an empty `Graph` object. Then, we add a **boundary input vertex** (`in_v`) at qubit 0 and row 0, with the `ground=True` flag to indicate it's classical. Next, we place two **Z-spiders** (`z1` and `z2`) at subsequent rows, also marked as classical. Finally, we add a **boundary output vertex** (`out_v`) at row 3. These vertices are connected sequentially using edges, forming a simple chain: input → Z → Z → output. We define the first and last vertices as the inputs and outputs of the graph, respectively. Mathematically, this structure resembles a classical signal passing through two identity operations (Z-spiders without phase), with no quantum entanglement or phase manipulation involved. It forms the simplest backbone for understanding classical data flow in ZX-diagrams.


In [ ]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import numpy as np

def create_measurement_circuit():
    """Demo 1: Basic measurement with classical discard using grounds"""
    g = GraphS()

    # Create 2-qubit system
    in1 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0)
    in2 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0)
    out1 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=4)
    out2 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=4)

    g.set_inputs([in1, in2])
    g.set_outputs([out1, out2])

    # Add Hadamard on first qubit
    h1 = g.add_vertex(VertexType.X, qubit=0, row=1, phase=1)  # H gate
    g.add_edge((in1, h1), EdgeType.SIMPLE)

    # Add CNOT between qubits
    z_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=2)
    x_targ = g.add_vertex(VertexType.X, qubit=1, row=2)
    g.add_edge((h1, z_ctrl), EdgeType.SIMPLE)
    g.add_edge((in2, x_targ), EdgeType.SIMPLE)
    g.add_edge((z_ctrl, x_targ), EdgeType.HADAMARD)

    # Measurement on first qubit - this becomes classical
    meas = g.add_vertex(VertexType.Z, qubit=0, row=3)
    g.add_edge((z_ctrl, meas), EdgeType.SIMPLE)
    g.set_ground(meas, True)  # Ground indicates classical measurement/discard

    # Second qubit continues quantum
    g.add_edge((x_targ, out2), EdgeType.SIMPLE)

    return g

def create_conditional_circuit():
    """Demo 2: Conditional operations based on measurement outcomes"""
    g = GraphS()

    # 3-qubit system: measure qubit 0, conditionally operate on qubit 1
    inputs = []
    outputs = []
    for i in range(3):
        inp = g.add_vertex(VertexType.BOUNDARY, qubit=i, row=0)
        out = g.add_vertex(VertexType.BOUNDARY, qubit=i, row=6)
        inputs.append(inp)
        outputs.append(out)

    g.set_inputs(inputs)
    g.set_outputs(outputs)

    # Prepare entangled state on qubits 0 and 1
    h0 = g.add_vertex(VertexType.X, qubit=0, row=1, phase=1)
    g.add_edge((inputs[0], h0), EdgeType.SIMPLE)

    # CNOT: qubit 0 -> qubit 1
    z0 = g.add_vertex(VertexType.Z, qubit=0, row=2)
    x1 = g.add_vertex(VertexType.X, qubit=1, row=2)
    g.add_edge((h0, z0), EdgeType.SIMPLE)
    g.add_edge((inputs[1], x1), EdgeType.SIMPLE)
    g.add_edge((z0, x1), EdgeType.HADAMARD)

    # Measure qubit 0
    meas0 = g.add_vertex(VertexType.Z, qubit=0, row=3)
    g.add_edge((z0, meas0), EdgeType.SIMPLE)
    g.set_ground(meas0, True)  # Classical measurement result

    # Conditional Z rotation on qubit 2 based on measurement
    cond_z = g.add_vertex(VertexType.Z, qubit=2, row=4, phase=0.5)  # π/2 rotation
    g.add_edge((inputs[2], cond_z), EdgeType.SIMPLE)

    # The measurement result influences the conditional gate
    # In a real implementation, this would be controlled classically

    # Final operations
    g.add_edge((x1, outputs[1]), EdgeType.SIMPLE)
    g.add_edge((cond_z, outputs[2]), EdgeType.SIMPLE)

    return g

def create_teleportation_circuit():
    """Demo 3: Quantum teleportation with classical communication"""
    g = GraphS()

    # 3-qubit teleportation: Alice has qubits 0,1; Bob has qubit 2
    inputs = []
    outputs = []
    for i in range(3):
        inp = g.add_vertex(VertexType.BOUNDARY, qubit=i, row=0)
        out = g.add_vertex(VertexType.BOUNDARY, qubit=i, row=8)
        inputs.append(inp)
        outputs.append(out)

    g.set_inputs(inputs)
    g.set_outputs(outputs)

    # Prepare Bell pair between qubits 1 and 2
    h1 = g.add_vertex(VertexType.X, qubit=1, row=1, phase=1)
    g.add_edge((inputs[1], h1), EdgeType.SIMPLE)

    z1 = g.add_vertex(VertexType.Z, qubit=1, row=2)
    x2 = g.add_vertex(VertexType.X, qubit=2, row=2)
    g.add_edge((h1, z1), EdgeType.SIMPLE)
    g.add_edge((inputs[2], x2), EdgeType.SIMPLE)
    g.add_edge((z1, x2), EdgeType.HADAMARD)

    # Bell measurement on Alice's qubits (0 and 1)
    # First: CNOT from qubit 0 to qubit 1
    z0_bell = g.add_vertex(VertexType.Z, qubit=0, row=3)
    x1_bell = g.add_vertex(VertexType.X, qubit=1, row=3)
    g.add_edge((inputs[0], z0_bell), EdgeType.SIMPLE)
    g.add_edge((z1, x1_bell), EdgeType.SIMPLE)
    g.add_edge((z0_bell, x1_bell), EdgeType.HADAMARD)

    # Hadamard on qubit 0
    h0_bell = g.add_vertex(VertexType.X, qubit=0, row=4, phase=1)
    g.add_edge((z0_bell, h0_bell), EdgeType.SIMPLE)

    # Measurements (become classical)
    meas0 = g.add_vertex(VertexType.Z, qubit=0, row=5)
    meas1 = g.add_vertex(VertexType.Z, qubit=1, row=5)
    g.add_edge((h0_bell, meas0), EdgeType.SIMPLE)
    g.add_edge((x1_bell, meas1), EdgeType.SIMPLE)

    # Ground the measurement results (classical information)
    g.set_ground(meas0, True)
    g.set_ground(meas1, True)

    # Bob's corrections based on classical bits
    # X correction (if meas1 = 1)
    x_corr = g.add_vertex(VertexType.X, qubit=2, row=6)
    g.add_edge((x2, x_corr), EdgeType.SIMPLE)

    # Z correction (if meas0 = 1)
    z_corr = g.add_vertex(VertexType.Z, qubit=2, row=7)
    g.add_edge((x_corr, z_corr), EdgeType.SIMPLE)
    g.add_edge((z_corr, outputs[2]), EdgeType.SIMPLE)

    return g

def create_error_correction_circuit():
    """Demo 4: Simple error correction with syndrome measurement"""
    g = GraphS()

    # 5-qubit system: 3 data qubits + 2 ancilla for syndrome
    inputs = []
    outputs = []
    for i in range(5):
        inp = g.add_vertex(VertexType.BOUNDARY, qubit=i, row=0)
        out = g.add_vertex(VertexType.BOUNDARY, qubit=i, row=8)
        inputs.append(inp)
        outputs.append(out)

    g.set_inputs(inputs)
    g.set_outputs(outputs)

    # Initialize ancilla qubits in |+⟩ state
    h3 = g.add_vertex(VertexType.X, qubit=3, row=1, phase=1)
    h4 = g.add_vertex(VertexType.X, qubit=4, row=1, phase=1)
    g.add_edge((inputs[3], h3), EdgeType.SIMPLE)
    g.add_edge((inputs[4], h4), EdgeType.SIMPLE)

    # Syndrome extraction: parity checks
    # First parity check: qubits 0, 1, 3
    z0_s1 = g.add_vertex(VertexType.Z, qubit=0, row=2)
    x3_s1 = g.add_vertex(VertexType.X, qubit=3, row=2)
    g.add_edge((inputs[0], z0_s1), EdgeType.SIMPLE)
    g.add_edge((h3, x3_s1), EdgeType.SIMPLE)
    g.add_edge((z0_s1, x3_s1), EdgeType.HADAMARD)

    z1_s1 = g.add_vertex(VertexType.Z, qubit=1, row=3)
    x3_s2 = g.add_vertex(VertexType.X, qubit=3, row=3)
    g.add_edge((inputs[1], z1_s1), EdgeType.SIMPLE)
    g.add_edge((x3_s1, x3_s2), EdgeType.SIMPLE)
    g.add_edge((z1_s1, x3_s2), EdgeType.HADAMARD)

    # Second parity check: qubits 1, 2, 4
    z1_s3 = g.add_vertex(VertexType.Z, qubit=1, row=4)
    x4_s1 = g.add_vertex(VertexType.X, qubit=4, row=4)
    g.add_edge((z1_s1, z1_s3), EdgeType.SIMPLE)
    g.add_edge((h4, x4_s1), EdgeType.SIMPLE)
    g.add_edge((z1_s3, x4_s1), EdgeType.HADAMARD)

    z2_s1 = g.add_vertex(VertexType.Z, qubit=2, row=5)
    x4_s2 = g.add_vertex(VertexType.X, qubit=4, row=5)
    g.add_edge((inputs[2], z2_s1), EdgeType.SIMPLE)
    g.add_edge((x4_s1, x4_s2), EdgeType.SIMPLE)
    g.add_edge((z2_s1, x4_s2), EdgeType.HADAMARD)

    # Measure ancilla qubits (syndrome bits)
    meas3 = g.add_vertex(VertexType.Z, qubit=3, row=6)
    meas4 = g.add_vertex(VertexType.Z, qubit=4, row=6)
    g.add_edge((x3_s2, meas3), EdgeType.SIMPLE)
    g.add_edge((x4_s2, meas4), EdgeType.SIMPLE)

    # Ground syndrome measurements (classical processing)
    g.set_ground(meas3, True)
    g.set_ground(meas4, True)

    # Data qubits continue (in practice, corrections would be applied)
    g.add_edge((z0_s1, outputs[0]), EdgeType.SIMPLE)
    g.add_edge((z1_s3, outputs[1]), EdgeType.SIMPLE)
    g.add_edge((z2_s1, outputs[2]), EdgeType.SIMPLE)

    return g

def demonstrate_ground_properties():
    """Demo 5: Explore ground vertex properties and hybrid detection"""
    g = GraphS()

    # Create a mixed circuit
    v1 = g.add_vertex(VertexType.Z, qubit=0, row=0)
    v2 = g.add_vertex(VertexType.X, qubit=1, row=0)
    v3 = g.add_vertex(VertexType.Z, qubit=2, row=0)

    # Set some vertices as grounded (classical)
    g.set_ground(v1, True)
    g.set_ground(v3, True)

    # Demonstrate ground-related methods
    print(f"Is v1 grounded? {g.is_ground(v1)}")
    print(f"Is v2 grounded? {g.is_ground(v2)}")
    print(f"Ground vertices: {g.grounds()}")
    print(f"Is hybrid circuit? {g.is_hybrid()}")

    return g

# Main demonstration
if __name__ == "__main__":
    # Demo 1: Basic measurement with classical discard
    g1 = create_measurement_circuit()
    zx.draw(g1, labels=True)


    # Demo 5: Ground properties exploration
    g5 = demonstrate_ground_properties()
    zx.draw(g5, labels=True)

Is v1 grounded? True
Is v2 grounded? False
Ground vertices: {0, 2}
Is hybrid circuit? True


This example demonstrates a simplified Quantum Phase Estimation (QPE)
circuit, focusing on how 'ground' parameters are used to represent the classical
 measurement outcomes of the "counting" register, which would then be classically
processed to extract the phase.

he circuit components include:
 - A 'counting' register (2 qubits for simplicity).
- A 'target' qubit prepared in an eigenvector state (here, a simple |+> for a Z eigenstate).
 - Hadamard gates on the counting qubits to put them into superposition.
 - Controlled-U gates (where U is a Z rotation in this simplified case) on the target qubit,
  controlled by the counting qubits.
 - Measurements on the counting qubits, with their vertices being 'grounded' to signify
   classical readout.

In [11]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math


# Step 1: Create a new ZX graph instance
g = GraphS()

# Step 2: Define input/output boundaries for counting and target qubits
# We'll use 2 qubits for the counting register (Q0, Q1) and 1 target qubit (Q2).
in_c0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Counting Qubit 0
in_c1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0) # Counting Qubit 1
in_t = g.add_vertex(VertexType.BOUNDARY, qubit=2, row=0)  # Target Qubit

out_c0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=7) # Output for Counting Qubit 0
out_c1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=7) # Output for Counting Qubit 1
out_t = g.add_vertex(VertexType.BOUNDARY, qubit=2, row=7)  # Output for Target Qubit

g.set_inputs([in_c0, in_c1, in_t])
g.set_outputs([out_c0, out_c1, out_t])

# Step 3: Initialize counting qubits to |+> (Hadamard decomposition)
# Q0: Hadamard
h_c0_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0)
h_c0_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0)
g.add_edge((in_c0, h_c0_z), EdgeType.SIMPLE)
g.add_edge((h_c0_z, h_c0_x), EdgeType.HADAMARD)
q_c0_curr = h_c0_x

# Q1: Hadamard
h_c1_z = g.add_vertex(VertexType.Z, qubit=1, row=1, phase=0)
h_c1_x = g.add_vertex(VertexType.X, qubit=1, row=1.1, phase=0)
g.add_edge((in_c1, h_c1_z), EdgeType.SIMPLE)
g.add_edge((h_c1_z, h_c1_x), EdgeType.HADAMARD)
q_c1_curr = h_c1_x

# Step 4: Prepare target qubit in an eigenstate (e.g., |+> for a Z-rotation U)
# Hadamard on target qubit
h_t_z = g.add_vertex(VertexType.Z, qubit=2, row=1, phase=0)
h_t_x = g.add_vertex(VertexType.X, qubit=2, row=1.1, phase=0)
g.add_edge((in_t, h_t_z), EdgeType.SIMPLE)
g.add_edge((h_t_z, h_t_x), EdgeType.HADAMARD)
q_t_curr = h_t_x

# Step 5: Controlled-U gates (U = Rz(theta) in this example, theta = pi/2)
# The phase to be estimated is theta. The controlled-U operations apply U^(2^k)
# For simplicity, we'll use CNOTs with Z-phases on the target qubit,
# acting as controlled-Rz.

# CNOT (Q0, Q2) with Z-phase on Q2 (for U^(2^0))
# This simulates controlled-Rz(theta * 2^0) = controlled-Rz(theta)
cu0_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=2)
cu0_targ_x = g.add_vertex(VertexType.X, qubit=2, row=2) # X-spider for controlled phase application
cu0_targ_z = g.add_vertex(VertexType.Z, qubit=2, row=2.1, phase=math.pi / 2) # Z-spider with phase

g.add_edge((q_c0_curr, cu0_ctrl), EdgeType.SIMPLE)
g.add_edge((q_t_curr, cu0_targ_x), EdgeType.SIMPLE)
g.add_edge((cu0_ctrl, cu0_targ_x), EdgeType.HADAMARD) # CNOT part
g.add_edge((cu0_targ_x, cu0_targ_z), EdgeType.SIMPLE) # Apply Z-phase on target after CNOT

q_c0_curr = cu0_ctrl
q_t_curr = cu0_targ_z


# CNOT (Q1, Q2) with Z-phase on Q2 (for U^(2^1))
# This simulates controlled-Rz(theta * 2^1) = controlled-Rz(2*theta)
cu1_ctrl = g.add_vertex(VertexType.Z, qubit=1, row=3)
cu1_targ_x = g.add_vertex(VertexType.X, qubit=2, row=3)
cu1_targ_z = g.add_vertex(VertexType.Z, qubit=2, row=3.1, phase=math.pi) # Phase is 2 * (pi/2) = pi

g.add_edge((q_c1_curr, cu1_ctrl), EdgeType.SIMPLE)
g.add_edge((q_t_curr, cu1_targ_x), EdgeType.SIMPLE) # Target qubit continues from previous CNOT
g.add_edge((cu1_ctrl, cu1_targ_x), EdgeType.HADAMARD) # CNOT part
g.add_edge((cu1_targ_x, cu1_targ_z), EdgeType.SIMPLE) # Apply Z-phase on target after CNOT

q_c1_curr = cu1_ctrl
q_t_curr = cu1_targ_z


# Step 6: Measurements on Counting Qubits & Grounding
# After the controlled-U operations, the counting qubits are measured.
# We ground these measurement vertices, indicating their outcomes are classical.
meas_c0 = g.add_vertex(VertexType.Z, qubit=0, row=4)
meas_c1 = g.add_vertex(VertexType.Z, qubit=1, row=4)

g.add_edge((q_c0_curr, meas_c0), EdgeType.SIMPLE)
g.add_edge((q_c1_curr, meas_c1), EdgeType.SIMPLE)

# Grounding the measurement outcomes of the counting register.
# These classical bits would then be used in an Inverse Quantum Fourier Transform (IQFT)
# and classical post-processing to estimate the phase.
g.set_ground(meas_c0, True)
g.set_ground(meas_c1, True)

# Step 7: Connect all current qubit paths to their respective output boundaries.
# The target qubit remains quantum until the end.
g.add_edge((q_t_curr, out_t), EdgeType.SIMPLE)
g.add_edge((meas_c0, out_c0), EdgeType.SIMPLE)
g.add_edge((meas_c1, out_c1), EdgeType.SIMPLE)


# Visualize the initial complex QPE circuit
zx.draw(g)

# Step 8: Simplify the graph using ZX-calculus reduction rules.
# The 'full_reduce' function will apply comprehensive rewrite rules.
# Because the counting qubit measurements are grounded, the simplification process
# will be able to effectively "collapse" or remove parts of the graph related
# to these qubits once their information is classically resolved.
zx.simplify.full_reduce(g)

# Step 9: Visualize the reduced diagram.
# Observe how the graph simplifies, particularly the paths involving the grounded
# counting qubits.
zx.draw(g)


 This example showcases a quantum state verification process where
 an ancilla qubit is used to check a property of a primary qubit's state. The
 measurement of the ancilla qubit provides a classical "verification bit," and
we use the 'ground' parameter to model this classical readout.

The circuit involves:
- A 'primary' qubit in some initial state (e.g., |0> or a superposition).
 - An 'ancilla' qubit initialized to |0>.
- Entanglement between the primary and ancilla qubits (e.g., CNOT).
- A measurement of the ancilla qubit, which is then 'grounded'. This outcome
  is the classical verification result.
- The primary qubit continues its path (its state might be altered by the interaction).

In [12]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math



# Step 1: Create a new ZX graph instance
g = GraphS()

# Step 2: Define input/output boundaries for primary and ancilla qubits
in_primary = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Primary Qubit
in_ancilla = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0) # Ancilla Qubit

out_primary = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=4) # Output for Primary Qubit
out_ancilla = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=4) # Output for Ancilla Qubit

g.set_inputs([in_primary, in_ancilla])
g.set_outputs([out_primary, out_ancilla])

# Step 3: Prepare the primary qubit in a state to be verified
# Let's say we prepare it in |+> state, implying we're verifying if it's in X-basis
h_primary_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0)
h_primary_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0)
g.add_edge((in_primary, h_primary_z), EdgeType.SIMPLE)
g.add_edge((h_primary_z, h_primary_x), EdgeType.HADAMARD)
q_primary_curr = h_primary_x
q_ancilla_curr = in_ancilla # Ancilla starts as |0>

# Step 4: Entangle primary and ancilla for verification
# We'll use a CNOT where primary is control, ancilla is target.
# If primary is |0>, ancilla stays |0>. If primary is |1>, ancilla flips to |1>.
# This effectively copies a property of the primary to the ancilla.
cnot_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=2)
cnot_targ = g.add_vertex(VertexType.X, qubit=1, row=2)

g.add_edge((q_primary_curr, cnot_ctrl), EdgeType.SIMPLE)
g.add_edge((q_ancilla_curr, cnot_targ), EdgeType.SIMPLE)
g.add_edge((cnot_ctrl, cnot_targ), EdgeType.HADAMARD)

q_primary_curr = cnot_ctrl # Primary qubit continues after being a control
q_ancilla_curr = cnot_targ # Ancilla qubit has been flipped conditionally

# Step 5: Measure the ancilla qubit & Ground its outcome
# The measurement outcome of the ancilla provides the classical verification result.
# For instance, if ancilla is 0, the primary might be considered "verified" for a certain property.
meas_ancilla = g.add_vertex(VertexType.Z, qubit=1, row=3)
g.add_edge((q_ancilla_curr, meas_ancilla), EdgeType.SIMPLE)

# Grounding the ancilla measurement: This signals that its quantum information
# has been converted into a classical bit, which is our verification outcome.
g.set_ground(meas_ancilla, True)

# Step 6: Connect qubits to their respective output boundaries
g.add_edge((q_primary_curr, out_primary), EdgeType.SIMPLE)
g.add_edge((meas_ancilla, out_ancilla), EdgeType.SIMPLE)


# Visualize the initial complex state verification circuit
zx.draw(g)

# Step 7: Simplify the graph using ZX-calculus reduction rules.
# The 'full_reduce' function will apply comprehensive rewrite rules.
# Because the ancilla measurement is grounded, PyZX can simplify the graph
# by accounting for the classical resolution of that qubit's information.
zx.simplify.full_reduce(g)

# Step 8: Visualize the reduced diagram.
# Observe the simplification. The grounded ancilla effectively allows its path
# to be removed or collapsed, demonstrating how the classical extraction of
# information leads to a more compact ZX-diagram.
zx.draw(g)



## PyZX Hybrid Quantum-Classical Circuit Extension
================================================

This extended demo explores additional hybrid quantum-classical circuit functionality
using the 'ground' feature in PyZX. The original framework is preserved while adding
new circuits for classical fanout, hybrid multiplexing, and measurement-controlled routing.
"""

Let’s see how to build more advanced hybrid quantum-classical circuits using PyZX. This extended example demonstrates three hybrid patterns: classical fanout, hybrid multiplexing, and measurement-controlled routing—each showing different interactions between classical control and quantum data. These circuits use ground nodes to represent classical (non-coherent) values alongside quantum wires in ZX-diagrams.

## 1. Classical Fanout Circuit
This demonstrates copying a classical bit (from a single classical input) to multiple classical outputs—a non-quantum operation allowed in classical logic but forbidden in quantum circuits (due to the no-cloning theorem).

A classical input is created with ground=True.

Two Z-spiders (also grounded) act as fan-out junctions.

Two classical outputs receive the copied bits.

Ground nodes: All except the boundary are marked classical.

ℹ️ Uses g.grounds() to count and verify grounded vertices. Useful to distinguish classical logic within hybrid systems.

## 2. Hybrid Multiplexer Circuit
Here, a classical selector bit determines which of two quantum inputs is routed forward—a simple multiplexing operation based on classical control.

A classical selector node is marked with ground=True.

Two quantum inputs enter on different wires.

A classical control Z node influences two quantum gates: mux_z and mux_x (Z and X spiders).

Both gate outputs merge at a single quantum output.

💡 Demonstrates hybrid control flow—classical bits conditionally triggering operations on quantum wires.

## 3. Measurement-Controlled Routing
This shows a quantum measurement creating a classical control, which is then used to route a quantum state to different output ports.

A quantum input goes into a measuring Z-spider (treated as classical via ground=True).

Its outcome drives a classical control Z spider.

The input is partially preserved to a qmid node (representing unmeasured path).

Depending on the control value, either out1 or out2 is selected.

🧠 This simulates a mid-circuit measurement, where the measurement outcome affects downstream circuit behavior—a common feature in NISQ-era hybrid algorithms.

In [13]:
import pyzx as zx
from fractions import Fraction
from typing import List, Tuple, Dict, Set

class HybridCircuitExtension:
    def __init__(self):
        print("=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===\n")

    def classical_fanout(self):
        """Demonstrate classical fan-out: copying a classical bit to multiple locations."""
        print("7. Classical Fanout Circuit")
        print("-" * 30)

        g = zx.Graph()

        # Classical input bit
        classical_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0, ground=True)

        # Fan-out nodes (e.g., copy to 2 locations)
        fan1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)
        fan2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical outputs
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=2, ground=True)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=2, ground=True)

        g.add_edge((classical_input, fan1))
        g.add_edge((classical_input, fan2))
        g.add_edge((fan1, out1))
        g.add_edge((fan2, out2))

        g.set_inputs((classical_input,))
        g.set_outputs((out1, out2))

        print(f"Fanout circuit created with {g.num_vertices()} vertices")
        print(f"Ground (classical) vertices: {len(g.grounds())}")
        print(f"Graph is hybrid: {g.is_hybrid()}\n")

    def hybrid_multiplexer(self):
        """Demonstrate a hybrid multiplexer circuit: select quantum data line via classical control."""
        print("8. Hybrid Multiplexer Circuit")
        print("-" * 30)

        g = zx.Graph()

        # Classical selector (ground)
        sel = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0, ground=True)

        # Quantum inputs
        q1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)
        q2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=0)

        # Control logic to switch inputs
        ctrl = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Multiplexer logic: apply Z or X gate based on selector
        mux_z = g.add_vertex(zx.VertexType.Z, qubit=1, row=2)
        mux_x = g.add_vertex(zx.VertexType.X, qubit=2, row=2)

        # Quantum output
        out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=3, row=3)

        g.add_edge((sel, ctrl))
        g.add_edge((ctrl, mux_z))
        g.add_edge((ctrl, mux_x))
        g.add_edge((q1, mux_z))
        g.add_edge((q2, mux_x))
        g.add_edge((mux_z, out))
        g.add_edge((mux_x, out))

        g.set_inputs((sel, q1, q2))
        g.set_outputs((out,))

        print(f"Multiplexer circuit created with {g.num_vertices()} vertices")
        print(f"Hybrid: {g.is_hybrid()}")
        print(f"Ground vertices: {g.grounds()}\n")

    def measurement_controlled_routing(self):
        """Use measurement result to route a quantum state to different output paths."""
        print("9. Measurement-Controlled Routing")
        print("-" * 30)

        g = zx.Graph()

        # Quantum input
        qin = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Mid-circuit measurement (creates classical control bit)
        measure = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical control node
        ctrl = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)

        # Routing logic: if ctrl==0 -> out1; if ctrl==1 -> out2
        qmid = g.add_vertex(zx.VertexType.Z, qubit=0, row=3)
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=4)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=4)

        g.add_edge((qin, measure))
        g.add_edge((measure, ctrl))
        g.add_edge((measure, qmid))
        g.add_edge((ctrl, out1))
        g.add_edge((qmid, out2))

        g.set_inputs((qin,))
        g.set_outputs((out1, out2))

        print(f"Measurement routing circuit created")
        print(f"Hybrid: {g.is_hybrid()} | Ground vertices: {g.grounds()}\n")


if __name__ == "__main__":
    ext = HybridCircuitExtension()
    ext.classical_fanout()
    ext.hybrid_multiplexer()
    ext.measurement_controlled_routing()


=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===

7. Classical Fanout Circuit
------------------------------
Fanout circuit created with 5 vertices
Ground (classical) vertices: 5
Graph is hybrid: True

8. Hybrid Multiplexer Circuit
------------------------------
Multiplexer circuit created with 7 vertices
Hybrid: True
Ground vertices: {0, 3}

9. Measurement-Controlled Routing
------------------------------
Measurement routing circuit created
Hybrid: True | Ground vertices: {1, 2}



Let’s explore more advanced hybrid quantum-classical circuit structures in PyZX, highlighting how classical information (from measurements or inputs) can dynamically affect quantum processing. The following examples extend hybrid circuit design into feedback control, fanout with multi-qubit gates, and cascading adaptive behavior—important for realistic quantum-classical algorithms.

## 1. Classical Feedback Loop
This circuit models a feedback-controlled quantum operation, where a measured quantum state informs a future operation on the same qubit.

A quantum input is measured early (meas) using a grounded Z-spider.

The measurement outcome is passed to a classical processor (classical_proc, also grounded).

A conditional quantum gate (feedback_gate) is applied using that classical value.

A final measurement captures the result.

🔁 The key loop: quantum → classical (meas) → feedback gate → quantum.
✅ Grounded nodes (ground=True) represent classical data from measurement and logic.

## 2. Classical Fan-Out with Conditional Gates
Here, a single classical input controls two different quantum operations, mimicking parallel classical control logic.

A classical control bit is introduced via a boundary vertex with ground=True.

That control is fanned out to influence:

A Pauli-X gate on q1

A Pauli-Z gate on q2

Quantum paths are independent, but classically synchronized.

📌 Highlights multi-target classical control, showing that classical signals can coordinate actions across different quantum channels in hybrid circuits.

## 3. Adaptive Measurement Cascade
This circuit builds a cascading control flow: each quantum operation is conditionally executed based on successive measurements.

The quantum input flows through:

A first measurement → classical control → conditional gate

Then another measurement → classical control → second conditional gate

Each measurement result affects the next quantum operation.

📈 This reflects dynamic/adaptive behavior, like real-time decision-making or branching logic, based on measurement outcomes.


## Extended PyZX Hybrid Quantum-Classical Circuit Demo
==================================================

This extension explores additional functionality of hybrid circuits
including feedback control, classical bit fanout, dynamic reconfiguration,
and mixed measurement-feedback loops in the PyZX ZX-calculus framework.


In [14]:
import pyzx as zx
from fractions import Fraction

class ExtendedHybridCircuitDemo:
    def __init__(self):
        print("=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===\n")

    def classical_feedback_loop(self):
        """Demonstrates classical feedback control with a loop."""
        print("7. Classical Feedback Loop")
        print("-" * 30)

        g = zx.Graph()

        # Quantum input
        q_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Initial measurement
        meas = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical processing and feedback gate
        classical_proc = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)
        feedback_gate = g.add_vertex(zx.VertexType.X, qubit=0, row=3, phase=Fraction(1))

        # Final measurement
        final_meas = g.add_vertex(zx.VertexType.Z, qubit=0, row=4, ground=True)
        q_out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=5)

        # Connect components
        g.add_edge((q_in, meas))
        g.add_edge((meas, classical_proc))
        g.add_edge((classical_proc, feedback_gate))
        g.add_edge((feedback_gate, final_meas))
        g.add_edge((final_meas, q_out))

        g.set_inputs((q_in,))
        g.set_outputs((q_out,))

        print(f"Vertices: {g.num_vertices()}, Grounds: {len(g.grounds())}, Is hybrid: {g.is_hybrid()}\n")

    def classical_fanout_and_condition(self):
        """Demonstrates classical fan-out control over multiple qubits."""
        print("8. Classical Fan-Out and Conditional Gates")
        print("-" * 30)

        g = zx.Graph()

        # Classical control input
        c_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=0, ground=True)
        control = g.add_vertex(zx.VertexType.Z, qubit=2, row=1, ground=True)

        # Two quantum inputs
        q1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)
        q2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)

        # Two controlled gates
        gate1 = g.add_vertex(zx.VertexType.X, qubit=0, row=2, phase=Fraction(1))
        gate2 = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, phase=Fraction(1))

        # Connect control to both gates
        g.add_edge((c_in, control))
        g.add_edge((control, gate1))
        g.add_edge((control, gate2))

        # Connect quantum flow
        g.add_edge((q1, gate1))
        g.add_edge((q2, gate2))

        # Outputs
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=3)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=3)
        g.add_edge((gate1, out1))
        g.add_edge((gate2, out2))

        g.set_inputs((q1, q2, c_in))
        g.set_outputs((out1, out2))

        print(f"Fanout control circuit created. Grounds: {len(g.grounds())}, Hybrid: {g.is_hybrid()}\n")

    def adaptive_measurement_cascade(self):
        """Models a dynamic circuit with cascading measurements."""
        print("9. Adaptive Measurement Cascade")
        print("-" * 30)

        g = zx.Graph()

        q = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # First measurement and conditional gate
        meas1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)
        ctrl1 = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)
        gate1 = g.add_vertex(zx.VertexType.X, qubit=0, row=3, phase=Fraction(1))

        # Second measurement and conditional gate
        meas2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=4, ground=True)
        ctrl2 = g.add_vertex(zx.VertexType.Z, qubit=2, row=5, ground=True)
        gate2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=6, phase=Fraction(1))

        out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=7)

        g.add_edge((q, meas1))
        g.add_edge((meas1, ctrl1))
        g.add_edge((ctrl1, gate1))
        g.add_edge((gate1, meas2))
        g.add_edge((meas2, ctrl2))
        g.add_edge((ctrl2, gate2))
        g.add_edge((gate2, out))

        g.set_inputs((q,))
        g.set_outputs((out,))

        print(f"Cascading feedback circuit created. Grounds: {len(g.grounds())}, Is hybrid: {g.is_hybrid()}\n")

if __name__ == "__main__":
    demo = ExtendedHybridCircuitDemo()
    demo.classical_feedback_loop()
    demo.classical_fanout_and_condition()
    demo.adaptive_measurement_cascade()

=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===

7. Classical Feedback Loop
------------------------------
Vertices: 6, Grounds: 3, Is hybrid: True

8. Classical Fan-Out and Conditional Gates
------------------------------
Fanout control circuit created. Grounds: 2, Hybrid: True

9. Adaptive Measurement Cascade
------------------------------
Cascading feedback circuit created. Grounds: 4, Is hybrid: True



##  1. Ground Parameter Fundamentals
This section lays the foundation:

It creates a few graph vertices representing quantum and classical gates.

Some vertices are marked as ground, meaning they're classical.

A vertex’s ground status is examined.

Copies and adjoints of the graph are made to demonstrate that the ground status is preserved, which is key when transforming circuits (e.g., during optimization).



##  2. Quantum Error Correction with Syndrome Extraction
Here, the bit-flip code is implemented:

Data qubits are encoded.

Ancilla qubits perform syndrome extraction via entangling gates.

Measurements (as ground vertices) extract error syndromes.

A classical error decoder processes these syndromes.

Quantum correction gates apply fixes based on classical outputs.



## 3. Adaptive Quantum Algorithms
This section demonstrates dynamic circuits:

A qubit is prepared in superposition.

It undergoes measurement mid-circuit (first adaptive step).

Depending on the outcome, a branch of logic is selected:

If measured 0, do nothing.

If measured 1, apply a complex rotation sequence.

A second measurement further adapts computation.

Finally, classical postprocessing computes an output.



##  4. Quantum Machine Learning (QML)
This part builds a hybrid ML pipeline:

Classical features (e.g., image pixels) are input as ground vertices.

Preprocessing layers operate classically.

The features are encoded onto quantum qubits through parameterized gates.

Variational gates and entanglers build a quantum model.

Quantum outputs are measured and fed into:

Classical postprocessing (e.g., gradients and parameter updates).



## 5. Mid-Circuit Measurement Patterns
Two patterns are explored:

Sequential feed-forward: One qubit is measured, and based on the result, another qubit is manipulated.

Parallel measurements: Two qubits are independently measured, and their results are jointly processed in classical logic.


## 6. Ground Preservation Analysis
Here, the focus is on how well PyZX preserves classical information across graph manipulations:

It sets up a mix of quantum and classical vertices.

Edges are added across both types.

It prepares to analyze whether operations like graph copying, adjointing, or rewriting maintain the intended hybrid structure.


This example demonstrates a 3-qubit bit-flip error correction code
within the PyZX framework, specifically highlighting the use of 'ground' parameters for classical syndrome measurements.

The process involves:
1. Encoding a single logical qubit into three physical qubits.
2. Introducing a simulated bit-flip error on one of the physical qubits.
3. Performing syndrome measurements using ancilla qubits.
4. Crucially, 'grounding' the ancilla qubits after their measurement. This signifies that their quantum information has been classically observed and can be discarded, allowing for specific simplifications in the ZX diagram.
5. Showing the graph before and after simplification to observe the effect of grounding on the representation of the error correction circuit. The actual classical correction logic (applying an X gate based on syndrome) is conceptually enabled by the grounded measurements, and the graph simplification reflects this.

In [15]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math

# Step 1: Create a new ZX graph instance
g = GraphS()

# Step 2: Define input and output boundaries for physical and ancilla qubits.
# We'll use 3 physical qubits for the code (Q0, Q1, Q2) and 2 ancilla qubits (A0, A1)
# for syndrome measurement.

# Physical qubits for encoding/decoding
in_log = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Logical input
out_log = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=9) # Logical output

# Additional physical qubits used in the repetition code
q1_in = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0)
q2_in = g.add_vertex(VertexType.BOUNDARY, qubit=2, row=0)

q1_out = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=9)
q2_out = g.add_vertex(VertexType.BOUNDARY, qubit=2, row=9)

# Ancilla qubits for syndrome measurement
a0_in = g.add_vertex(VertexType.BOUNDARY, qubit=3, row=0)
a1_in = g.add_vertex(VertexType.BOUNDARY, qubit=4, row=0)

a0_out = g.add_vertex(VertexType.BOUNDARY, qubit=3, row=9)
a1_out = g.add_vertex(VertexType.BOUNDARY, qubit=4, row=9)

g.set_inputs([in_log, q1_in, q2_in, a0_in, a1_in])
g.set_outputs([out_log, q1_out, q2_out, a0_out, a1_out])

# Step 3: Encoding the logical qubit (3-qubit repetition code)
# Initial state: |psi>_L = |000> + |111>
# We start with the logical qubit at 'in_log' (acting as Q0 initially).
# CNOT(Q0, Q1), CNOT(Q0, Q2)
cn01_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=1) # Z-spider for CNOT control
cn01_targ = g.add_vertex(VertexType.X, qubit=1, row=1) # X-spider for CNOT target
g.add_edge((in_log, cn01_ctrl), EdgeType.SIMPLE)
g.add_edge((q1_in, cn01_targ), EdgeType.SIMPLE) # Q1 starts as |0>
g.add_edge((cn01_ctrl, cn01_targ), EdgeType.HADAMARD)

cn02_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=2)
cn02_targ = g.add_vertex(VertexType.X, qubit=2, row=2)
g.add_edge((cn01_ctrl, cn02_ctrl), EdgeType.SIMPLE) # Q0 path continues from previous CNOT
g.add_edge((q2_in, cn02_targ), EdgeType.SIMPLE) # Q2 starts as |0>
g.add_edge((cn02_ctrl, cn02_targ), EdgeType.HADAMARD)

# Connect Q1 and Q2 from their CNOT targets to the next stage
q1_encoded_out = cn01_targ
q2_encoded_out = cn02_targ
q0_encoded_out = cn02_ctrl # Q0 path continues from its second CNOT control

# Step 4: Introduce a simulated error (e.g., bit-flip on Qubit 1)
# An X gate represents a bit-flip error.
error_q1 = g.add_vertex(VertexType.X, qubit=1, row=3)
g.add_edge((q1_encoded_out, error_q1), EdgeType.SIMPLE) # Apply error to Qubit 1
q1_after_error = error_q1
q0_after_error = q0_encoded_out # Q0 is unaffected
q2_after_error = q2_encoded_out # Q2 is unaffected

# Step 5: Syndrome Measurement (using ancilla qubits)
# Measure s0 = Q0 XOR Q1 (for error on Q0 or Q1)
# Measure s1 = Q1 XOR Q2 (for error on Q1 or Q2)

# Syndrome Measurement 1: s0 = Q0 XOR Q1 -> Ancilla A0
synd0_ctrl_q0 = g.add_vertex(VertexType.Z, qubit=0, row=4)
synd0_ctrl_q1 = g.add_vertex(VertexType.Z, qubit=1, row=4)
synd0_targ_a0 = g.add_vertex(VertexType.X, qubit=3, row=4) # X-spider for Ancilla A0

g.add_edge((q0_after_error, synd0_ctrl_q0), EdgeType.SIMPLE)
g.add_edge((q1_after_error, synd0_ctrl_q1), EdgeType.SIMPLE)
g.add_edge((a0_in, synd0_targ_a0), EdgeType.SIMPLE) # Ancilla A0 starts as |0>

# CNOT (Q0, A0)
g.add_edge((synd0_ctrl_q0, synd0_targ_a0), EdgeType.HADAMARD)
# CNOT (Q1, A0)
g.add_edge((synd0_ctrl_q1, synd0_targ_a0), EdgeType.HADAMARD)

# Syndrome Measurement 2: s1 = Q1 XOR Q2 -> Ancilla A1
synd1_ctrl_q1 = g.add_vertex(VertexType.Z, qubit=1, row=5)
synd1_ctrl_q2 = g.add_vertex(VertexType.Z, qubit=2, row=5)
synd1_targ_a1 = g.add_vertex(VertexType.X, qubit=4, row=5) # X-spider for Ancilla A1

g.add_edge((synd0_ctrl_q1, synd1_ctrl_q1), EdgeType.SIMPLE) # Q1 path continues
g.add_edge((q2_after_error, synd1_ctrl_q2), EdgeType.SIMPLE)
g.add_edge((a1_in, synd1_targ_a1), EdgeType.SIMPLE) # Ancilla A1 starts as |0>

# CNOT (Q1, A1)
g.add_edge((synd1_ctrl_q1, synd1_targ_a1), EdgeType.HADAMARD)
# CNOT (Q2, A1)
g.add_edge((synd1_ctrl_q2, synd1_targ_a1), EdgeType.HADAMARD)

# Step 6: Grounding the Ancilla Qubits (Syndrome Measurement Outcomes)
# This is where 'ground' is critical. It signals that these ancilla qubits have
# been measured and their quantum information is now classical.
# This allows for specific ZX simplification rules that "absorb" or "discard"
# the classical branches, effectively simplifying the quantum part of the circuit.
g.set_ground(synd0_targ_a0, True) # Grounding Ancilla A0 after its measurement
g.set_ground(synd1_targ_a1, True) # Grounding Ancilla A1 after its measurement

# Preserve the paths for the physical qubits (Q0, Q1, Q2) that will be corrected/decoded
q0_pre_correct = synd0_ctrl_q0
q1_pre_correct = synd1_ctrl_q1
q2_pre_correct = synd1_ctrl_q2

# Step 7: Connect the physical qubits to the correction/decoding stage
# For demonstration, we'll assume a correction happens here. In a real scenario,
# classical logic based on the grounded syndrome bits (s0, s1) would apply
# an X gate to Q0, Q1, or Q2.
# To show the *effect* of correction in the graph simplification, we'll just connect
# these to the final decoding stage, trusting that grounding enables the simplification.
# If we wanted to explicitly model a conditional X gate, it would add more complexity
# to the graph, and the grounding would help in its simplification.

# We'll add some generic Z-spiders to indicate continuation, or simply connect to decoding
q0_continue = g.add_vertex(VertexType.Z, qubit=0, row=7)
q1_continue = g.add_vertex(VertexType.Z, qubit=1, row=7)
q2_continue = g.add_vertex(VertexType.Z, qubit=2, row=7)

g.add_edge((q0_pre_correct, q0_continue), EdgeType.SIMPLE)
g.add_edge((q1_pre_correct, q1_continue), EdgeType.SIMPLE)
g.add_edge((q2_pre_correct, q2_continue), EdgeType.SIMPLE)


# Step 8: Decoding the logical qubit
# This is the reverse of encoding: CNOT(Q0, Q1), CNOT(Q0, Q2)
# Here, the logical qubit will ideally be recovered on Qubit 0, if correction is successful.
cn01_decode_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=8)
cn01_decode_targ = g.add_vertex(VertexType.X, qubit=1, row=8)
g.add_edge((q0_continue, cn01_decode_ctrl), EdgeType.SIMPLE)
g.add_edge((q1_continue, cn01_decode_targ), EdgeType.SIMPLE)
g.add_edge((cn01_decode_ctrl, cn01_decode_targ), EdgeType.HADAMARD)

cn02_decode_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=8) # This will merge with previous Q0 Z-spider
cn02_decode_targ = g.add_vertex(VertexType.X, qubit=2, row=8)
g.add_edge((cn01_decode_ctrl, cn02_decode_ctrl), EdgeType.SIMPLE)
g.add_edge((q2_continue, cn02_decode_targ), EdgeType.SIMPLE)
g.add_edge((cn02_decode_ctrl, cn02_decode_targ), EdgeType.HADAMARD)


# Connect to output boundaries
g.add_edge((cn02_decode_ctrl, out_log), EdgeType.SIMPLE)
g.add_edge((cn01_decode_targ, q1_out), EdgeType.SIMPLE) # Q1 out
g.add_edge((cn02_decode_targ, q2_out), EdgeType.SIMPLE) # Q2 out

# Connect the grounded ancillas to their outputs
g.add_edge((synd0_targ_a0, a0_out), EdgeType.SIMPLE)
g.add_edge((synd1_targ_a1, a1_out), EdgeType.SIMPLE)


# Visualize the initial complex circuit
zx.draw(g)

# Step 9: Simplify the graph
# The 'full_reduce' function will apply ZX-calculus rewrite rules.
# Because the ancilla qubits are grounded, the simplification process will treat
# their classical outcomes as resolved, leading to a much cleaner graph for the
# remaining quantum operations. The ability to remove or collapse parts of the
# circuit due to grounded vertices is key here.
zx.simplify.full_reduce(g)

# Step 10: Visualize the reduced diagram
# Observe how the error and the syndrome measurements, due to the grounding,
# are largely absorbed into the simplified graph, leaving behind a more compact
# representation of the overall logical operation (ideally, the identity if correction works).
zx.draw(g)

This example demonstrates a simplified Quantum Machine Learning (QML)
circuit, focusing on how 'ground' parameters are used to represent the classical
measurement outcomes that would typically feed into a classical optimization loop.

To avoid the previous `AttributeError` caused by `zx.hsimplify.from_hypergraph_form`
returning `None`, this version constructs the circuit *without* `H_BOX` vertices.
Instead, Hadamards and other operations are directly built using `Z` and `X`
spiders and `HADAMARD` edges, which are the native elements for ZX-diagrams
and are directly compatible with `zx.simplify.full_reduce`.

In QML, a Variational Quantum Eigensolver (VQE) or Quantum Neural Network (QNN)
involves:
1. Encoding classical data into quantum states.
2. Applying a Parameterized Quantum Circuit (PQC).
3. Measuring observables to obtain classical outputs.
4. Using these classical outputs in a classical optimizer to update parameters.

Here, we simulate a small two-qubit QNN evaluating a single data point.
We will particularly highlight the 'grounding' of the final measurement,
signifying its quantum information is now a classical outcome.

In [16]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math


# Step 1: Create a new ZX graph instance
g = GraphS()

# Step 2: Define input and output boundaries for two qubits.
# Qubit 0 will carry the 'data' and be measured.
# Qubit 1 will act as an auxiliary qubit for entanglement in the QNN.
in0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Input for Data Qubit
in1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0) # Input for Auxiliary Qubit

out0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=6) # Output for Qubit 0's path
out1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=6) # Output for Qubit 1's path

g.set_inputs([in0, in1])
g.set_outputs([out0, out1])

# Step 3: Data Encoding Layer
h_encode_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0) # Z-spider for Hadamard
h_encode_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0) # X-spider (part of H decomposition)
g.add_edge((in0, h_encode_z), EdgeType.SIMPLE)
g.add_edge((h_encode_z, h_encode_x), EdgeType.HADAMARD) # Connect with Hadamard edge for H effect

# Step 4: Parameterized Quantum Circuit (PQC) Layer (Simplified QNN)
# This represents a small, fixed-parameter quantum layer.
# For this demo, we use arbitrary fixed phases.

# Rz rotation on Qubit 0 (e.g., parameter phi_1)
rz0_1 = g.add_vertex(VertexType.Z, qubit=0, row=2, phase=math.pi / 3) # Example phase
g.add_edge((h_encode_x, rz0_1), EdgeType.SIMPLE) # Connect from the H-effect

# Rz rotation on Qubit 1 (e.g., parameter phi_2)
rz1_1 = g.add_vertex(VertexType.Z, qubit=1, row=1, phase=math.pi / 2) # Example phase
g.add_edge((in1, rz1_1), EdgeType.SIMPLE)

# CNOT gate for entanglement between Qubit 0 (control) and Qubit 1 (target)
cnot_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=3)
cnot_targ = g.add_vertex(VertexType.X, qubit=1, row=3)

g.add_edge((rz0_1, cnot_ctrl), EdgeType.SIMPLE)
g.add_edge((rz1_1, cnot_targ), EdgeType.SIMPLE)
g.add_edge((cnot_ctrl, cnot_targ), EdgeType.HADAMARD)

# Another Rz rotation on Qubit 0 after entanglement (e.g., parameter phi_3)
rz0_2 = g.add_vertex(VertexType.Z, qubit=0, row=4, phase=math.pi / 6)
g.add_edge((cnot_ctrl, rz0_2), EdgeType.SIMPLE)


# Step 5: Observable Measurement and Grounding
# We simulate a Z-basis measurement on Qubit 0 at the end of the PQC.
meas_q0 = g.add_vertex(VertexType.Z, qubit=0, row=5) # Z-spider for Z-basis measurement on Qubit 0
g.add_edge((rz0_2, meas_q0), EdgeType.SIMPLE)

# Crucial for QML demo: Grounding the measurement vertex.
# This signifies that the quantum information from Qubit 0 has been measured,
# yielding a classical bit. This classical outcome is then passed to the
# classical optimization routine
# Grounding allows PyZX to perform specific simplifications as this quantum path
# is effectively 'resolved' into classical information.
g.set_ground(meas_q0, True)

# Connect the auxiliary qubit (Qubit 1) to its output boundary.
# This qubit might be discarded or used for other purposes.
g.add_edge((cnot_targ, out1), EdgeType.SIMPLE)

# Connect the grounded measurement vertex to its output boundary for completeness,
# though its quantum information is effectively classically extracted.
g.add_edge((meas_q0, out0), EdgeType.SIMPLE)

# Visualize the initial complex QML circuit
# This shows the full structure including encoding, the PQC layer, and the measurement.
zx.draw(g)

# Step 6: Simplify the graph using ZX-calculus reduction rules.
# Since there are no H_BOX vertices, `full_reduce` can be called directly.
# The 'full_reduce' function will apply comprehensive rewrite rules.
# Because 'meas_q0' is grounded, the simplification process will be able to
# effectively "collapse" or remove parts of the graph related to this qubit
# once its information is classically resolved, leading to a simpler diagram.
zx.simplify.full_reduce(g)

# Step 7: Visualize the reduced diagram.
# Observe how the graph simplifies, particularly the path involving the grounded
# measurement. This demonstrates how grounding helps to abstract away the quantum
# details of parts of the circuit once their information has become classical,
# which is relevant in hybrid quantum-classical algorithms like QML.
zx.draw(g)


This example extends the previous QML circuit to demonstrate advanced
mid-circuit measurement patterns within PyZX, focusing on how 'ground' parameters
facilitate the representation of classical control flow and processing.

## Key advanced patterns illustrated:
1. Sequential measurements with feed-forward control: The outcome of an early
 measurement (MCM1) conceptually influences subsequent quantum gates. We model
 this by grounding MCM1 and showing conditional paths.
2. Parallel measurements with joint classical processing: Multiple qubits are
 measured (MCM2_Q0, MCM2_Q1), and their outcomes are grounded, implying they
would be processed together classically (e.g., for syndrome extraction or feature vectors).
3. Correlation analysis between measurement outcomes: The presence of multiple
   grounded measurements across the circuit sets up a scenario where their
  classical outcomes could be correlated.

This QML circuit simulates:
 - Data Encoding
 - A first Parameterized Quantum Circuit (PQC) layer
- A Mid-Circuit Measurement (MCM1) on Qubit 0, which is grounded.
- Conditional gates on Qubit 0 based on MCM1's conceptual outcome (feed-forward).
- A second PQC layer.
- Parallel Mid-Circuit Measurements (MCM2) on Qubit 0 and Qubit 1, both grounded
  for joint classical processing and correlation analysis.

In [17]:
import pyzx as zx
from pyzx.graph.base import BaseGraph
from pyzx.graph.graph_s import GraphS
from pyzx.graph import VertexType, EdgeType
import math



# Step 1: Create a new ZX graph instance
g = GraphS()

# Step 2: Define input and output boundaries for two qubits.
# Qubit 0 will carry the 'data' and undergo sequential measurements.
# Qubit 1 will act as an auxiliary qubit and be involved in parallel measurements.
in0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=0) # Input for Data Qubit
in1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=0) # Input for Auxiliary Qubit

# Adjusted row for outputs to accommodate new gates/measurements
out0 = g.add_vertex(VertexType.BOUNDARY, qubit=0, row=12)
out1 = g.add_vertex(VertexType.BOUNDARY, qubit=1, row=12)

g.set_inputs([in0, in1])
g.set_outputs([out0, out1])

# Step 3: Data Encoding Layer (Hadamard decomposed to Z-X and Hadamard edge)
# Applies an H gate to Qubit 0 to put it in superposition.
h_encode_z = g.add_vertex(VertexType.Z, qubit=0, row=1, phase=0)
h_encode_x = g.add_vertex(VertexType.X, qubit=0, row=1.1, phase=0)
g.add_edge((in0, h_encode_z), EdgeType.SIMPLE)
g.add_edge((h_encode_z, h_encode_x), EdgeType.HADAMARD)
q0_curr = h_encode_x
q1_curr = in1 # Qubit 1 remains idle for now

# Step 4: First Parameterized Quantum Circuit (PQC) Layer
# Simple Rz rotation on Qubit 0 and CNOT with Qubit 1.
rz0_1 = g.add_vertex(VertexType.Z, qubit=0, row=2, phase=math.pi / 4) # Rz(pi/4)
g.add_edge((q0_curr, rz0_1), EdgeType.SIMPLE)
q0_curr = rz0_1

rz1_1 = g.add_vertex(VertexType.Z, qubit=1, row=2, phase=math.pi / 3) # Rz(pi/3)
g.add_edge((q1_curr, rz1_1), EdgeType.SIMPLE)
q1_curr = rz1_1

cnot1_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=3)
cnot1_targ = g.add_vertex(VertexType.X, qubit=1, row=3)
g.add_edge((q0_curr, cnot1_ctrl), EdgeType.SIMPLE)
g.add_edge((q1_curr, cnot1_targ), EdgeType.SIMPLE)
g.add_edge((cnot1_ctrl, cnot1_targ), EdgeType.HADAMARD)
q0_curr = cnot1_ctrl
q1_curr = cnot1_targ

# Step 5: Mid-Circuit Measurement 1 (MCM1) on Qubit 0 & Grounding (Sequential Control)
# This measurement simulates a decision point. Its outcome will classically
# influence which subsequent gate is applied.
mcm1_q0 = g.add_vertex(VertexType.Z, qubit=0, row=4)
g.add_edge((q0_curr, mcm1_q0), EdgeType.SIMPLE)

# Ground MCM1: This is critical for modeling feed-forward. It tells PyZX that
# this quantum information is classically observed.
g.set_ground(mcm1_q0, True)

# Step 6: Conditional Quantum Gates (Feed-Forward)
# We model two potential paths for Qubit 0 based on MCM1's conceptual outcome.
# In a real circuit, classical logic would select one path. In ZX, we represent both,
# and grounding helps in simplifying the graph to reflect this branching.
# Option 1: Apply Rz(pi/8) if MCM1 outcome is '0' (conceptually)
cond_gate1_q0 = g.add_vertex(VertexType.Z, qubit=0, row=5, phase=math.pi / 8)
g.add_edge((mcm1_q0, cond_gate1_q0), EdgeType.SIMPLE) # Connect from grounded measurement

# Option 2: Apply X gate if MCM1 outcome is '1' (conceptually)
# To show an alternative, we connect from the same logical point as cond_gate1_q0.
cond_gate2_q0 = g.add_vertex(VertexType.X, qubit=0, row=5)
g.add_edge((mcm1_q0, cond_gate2_q0), EdgeType.SIMPLE)

# Re-converge the paths for Qubit 0. This spider represents the state of Qubit 0
# after one of the conditional gates has been applied.
q0_reconverge = g.add_vertex(VertexType.Z, qubit=0, row=6)
g.add_edge((cond_gate1_q0, q0_reconverge), EdgeType.SIMPLE)
g.add_edge((cond_gate2_q0, q0_reconverge), EdgeType.SIMPLE)
q0_curr = q0_reconverge

# Step 7: Second Parameterized Quantum Circuit (PQC) Layer
# Another CNOT gate for more entanglement, involving Qubit 0 (now potentially modified
# by feed-forward) and Qubit 1.
cnot2_ctrl = g.add_vertex(VertexType.Z, qubit=0, row=7)
cnot2_targ = g.add_vertex(VertexType.X, qubit=1, row=7)
g.add_edge((q0_curr, cnot2_ctrl), EdgeType.SIMPLE)
g.add_edge((q1_curr, cnot2_targ), EdgeType.SIMPLE) # Qubit 1 continues its path
g.add_edge((cnot2_ctrl, cnot2_targ), EdgeType.HADAMARD)
q0_curr = cnot2_ctrl
q1_curr = cnot2_targ

# Step 8: Parallel Mid-Circuit Measurements (MCM2) & Joint Classical Processing
# Measure both Qubit 0 and Qubit 1. Their outcomes are grounded, indicating they
# are processed jointly by classical logic (e.g., for feature extraction for a classifier,
# or for analyzing quantum correlations).
mcm2_q0 = g.add_vertex(VertexType.Z, qubit=0, row=8)
mcm2_q1 = g.add_vertex(VertexType.Z, qubit=1, row=8)

g.add_edge((q0_curr, mcm2_q0), EdgeType.SIMPLE)
g.add_edge((q1_curr, mcm2_q1), EdgeType.SIMPLE)

# Ground both MCM2 measurements. This implies classical processing of both outcomes.
g.set_ground(mcm2_q0, True)
g.set_ground(mcm2_q1, True)

# Note: The 'grounded' vertices represent the points where quantum information
# becomes classical. Subsequent classical logic would take these two outcomes
# (e.g., bit values '0' or '1' from each measurement) and perform joint processing
# or correlation analysis. The ZX diagram will simplify based on this classicality.

# Step 9: Final connection to outputs (from grounded measurements)
# Connect the grounded measurement vertices to their respective output boundaries.
# The quantum information is already classically resolved at this point.
g.add_edge((mcm2_q0, out0), EdgeType.SIMPLE)
g.add_edge((mcm2_q1, out1), EdgeType.SIMPLE)


# Visualize the initial complex QML circuit
# This will show the full structure, including multiple measurement points,
# conditional paths, and parallel measurements.
zx.draw(g)

# Step 10: Simplify the graph using ZX-calculus reduction rules.
# The 'full_reduce' function will apply comprehensive rewrite rules.
# Because the measurement vertices are grounded, the simplification process will be able to
# effectively "collapse" or remove parts of the graph related to these qubits
# once their information is classically resolved, leading to a simpler diagram
# reflecting the effective quantum transformation that occurred.
zx.simplify.full_reduce(g)

# Step 11: Visualize the reduced diagram.
# Observe how the graph simplifies. The grounded measurements effectively
# decouple the quantum paths, allowing PyZX to significantly compact the
# representation, highlighting how classical outcomes influence the quantum
# circuit's effective structure.
zx.draw(g)